## Project Graham
 
 [Give an explanation]

In [2]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
# import snntorch as snn   

### Electrophysiology Background


In [ ]:
# Load in the curated EPhys Data as npz object 
fname = "data.npz"
data = np.load(fname, allow_pickle=True)

print(data.files)
print(f"neuron_data's type is: {type(data['neuron_data'])}")
# neuron_data = data["neuron_data"].item()

# Data
  # train: {neuron, [associated spike times]}
  # neuron_data:
  # redundant_pairs: literally that
  # fs: sampling rate

# Get the stuff
# neuron_data = data['neuron_data'].item()
# fs = data['fs'] #fs = frequency sample

# # For each spike time, normalize by sampling freq to seconds
# spike_times = data['train'].item()
# train = [times / fs for neuron_ID, times in spike_times.items()]
# We now have a list of neurons, each with associated spike times

['train', 'neuron_data', 'redundant_pairs', 'fs']
neuron_data's type is: <class 'numpy.ndarray'>


In [41]:
print(fname)

data.npz


In [ ]:
def get_population_fr(trains: list, bin_size=0.1, w=5):

    ''' Bin population frequencies by time '''

    trains = np.hstack(trains)
    rec_length = np.max(trains)
    bin_num = int(rec_length// bin_size) + 1
    bins = np.linspace(0, rec_length, bin_num)
    fr = np.histogram(trains, bins)[0] / bin_size
    fr_avg = np.convolve(fr, np.ones(w), 'same') / w
    return bins[1:], fr_avg

# Plot Setup
fig, axs = plt.subplots(1, 1, figsize=(16, 6))
axs1 = axs.twinx()

# Plot population
bins, fr_avg = get_population_fr(trains=train)
axs1.plot(bins, fr_avg, color='r', linewidth=3, alpha=0.5)

# Plot individual neurons
y = 0
for vv in train:
    axs.scatter(vv, [y]*len(vv), marker="|", c='k', s=4, alpha=0.7)
    y += 1

# Individual neuron plot
axs.set_title("Population Activity", fontsize=16)
axs.set_xlabel("Time (s)", fontsize=14)
axs.set_ylabel("Neuron Number", fontsize=14)
axs.xaxis.set_tick_params(labelsize=14)
axs.yaxis.set_tick_params(labelsize=14)

# Population plot
axs1.set_ylabel("Population Firing Rate (Hz)", fontsize=16, color='r')
axs1.set_xlabel("Time (s)", fontsize=14)
axs1.spines['right'].set_color('r')
axs1.spines['right'].set_linewidth(2)
axs1.tick_params(axis='y', colors='r')
axs1.yaxis.set_tick_params(labelsize=14)

In [ ]:
# Now that we know the plan, let's get the Spiking Neural Network going!